In [29]:
import requests
import pandas as pd
import os
import altair as alt

# Programa para acessar a nova API do Portal da Transparência: https://portaldatransparencia-h.cgu.gov.br/api-de-dados

Criado dia 2 de julho de 2018

Teste com Contratos do Poder Executivo Federal

É preciso informar: dataInicial, dataFinal e codigoOrgao (SIAFI)

O intervalo de consulta da API, neste tipo de dado, é só de um mês

Vou extrair os contratos de todos os meses de 2017 e com código com do Ministério da Saúde =  36000

In [39]:
ano = '2017'
dia_inicial = '01'
dia_final = '31'
codigoOrgao = '36000'

In [40]:
for num in range(1, 13):
    if num <= 9:
        mes = '0' + str(num)
    else:
        mes = str(num)
    
    print(mes)
    
    url = f'http://www.portaldatransparencia.gov.br/api-de-dados/contratos?dataInicial={dia_inicial}%2F{mes}%2F{ano}&dataFinal={dia_final}%2F{mes}%2F{ano}&codigoOrgao={codigoOrgao}&pagina=1'
    
    try: 
        r = requests.get(url)
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
        
    contratos = r.json()
    
    df_parcial = pd.DataFrame(contratos)
    
    if num == 1:
        raw_df = df_parcial
    elif num > 1:
        raw_df = raw_df.append(df_parcial)
        
    

01
02
03
04
05
06
07
08
09
10
11
12


In [41]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 14
Data columns (total 13 columns):
contrato              0 non-null object
dataAssinatura        174 non-null object
dataFimVigencia       8 non-null object
dataInicioVigencia    174 non-null object
dataPublicacaoDOU     174 non-null object
dimCompra             174 non-null object
fornecedor            174 non-null object
id                    174 non-null int64
modalidadeCompra      0 non-null object
situacaoContrato      0 non-null object
unidadeGestora        174 non-null object
valorFinalCompra      174 non-null float64
valorInicialCompra    174 non-null float64
dtypes: float64(2), int64(1), object(10)
memory usage: 19.0+ KB


Formato com duas casas decimais

In [42]:
pd.options.display.float_format = '{:.2f}'.format

Definir diretórios de entrada e saída

In [43]:
input_dir = os.path.join(os.getcwd(), "input")
output_dir = os.path.join(os.getcwd(), "output")

In [44]:
df_contratos = raw_df 

Muda para formato de datas

In [45]:
df_contratos['dataAssinatura'] = pd.to_datetime(df_contratos['dataAssinatura'])
df_contratos['dataFimVigencia'] = pd.to_datetime(df_contratos['dataFimVigencia'])
df_contratos['dataInicioVigencia'] = pd.to_datetime(df_contratos['dataInicioVigencia'])
df_contratos['dataPublicacaoDOU'] = pd.to_datetime(df_contratos['dataPublicacaoDOU'])

In [46]:
df_contratos.reset_index().head()

,index,contrato,dataAssinatura,dataFimVigencia,dataInicioVigencia,dataPublicacaoDOU,dimCompra,fornecedor,id,modalidadeCompra,situacaoContrato,unidadeGestora,valorFinalCompra,valorInicialCompra
0,0,None,2017-10-07,NaT,2017-10-07,2017-09-08,"{'numero': '000122017', 'objeto': 'Objeto: For...","{'numeroInscricaoSocial': '', 'nome': 'CEB DIS...",870990,None,None,"{'codigo': '250110', 'nome': 'COORDENACAO-GERA...",1902587.26,1902587.26
1,1,None,2018-06-04,NaT,2018-06-04,2018-07-05,"{'numero': '000012018', 'objeto': 'Objeto: Con...","{'numeroInscricaoSocial': '', 'nome': 'AMAZONA...",871727,None,None,"{'codigo': '257027', 'nome': 'DISTRITO SANIT.E...",9600.00,9600.00
2,2,None,2017-10-23,NaT,2017-01-11,2017-11-23,"{'numero': '000152017', 'objeto': 'Objeto: Con...","{'numeroInscricaoSocial': '', 'nome': 'ENERGIS...",970151,None,None,"{'codigo': '250027', 'nome': 'NUCLEO ESTADUAL ...",85191.16,85191.16
3,3,None,2017-10-23,NaT,2017-01-11,2017-11-28,"{'numero': '000182017', 'objeto': 'Objeto: Con...","{'numeroInscricaoSocial': '', 'nome': 'ENERGIS...",1071248,None,None,"{'codigo': '250027', 'nome': 'NUCLEO ESTADUAL ...",81342.04,81342.04
4,4,None,2017-06-02,NaT,2017-06-02,2017-03-30,"{'numero': '000012017', 'objeto': 'Objeto: Con...","{'numeroInscricaoSocial': '', 'nome': 'EMPRESA...",1267590,None,None,"{'codigo': '250057', 'nome': 'INST. NACIONAL D...",411000.00,411000.00


Qual é o número de contratos?

In [50]:
len(df_contratos)

174

No total, quanto dinheiro foi pago?

In [54]:
"${:,.0f}".format(df_contratos.valorFinalCompra.sum())

'$42,159,169'

Quais são as outras estatísticas descritivas?

In [55]:
df_contratos.valorFinalCompra.describe().reset_index()

,index,valorFinalCompra
0,count,174.00
1,mean,242294.07
2,std,557709.74
3,min,0.01
4,25%,41697.92
5,50%,81342.04
6,75%,163981.62
7,max,3984000.00


Ainda é preciso converter os campos dimCompra, fornecedor e unidadeGestora para os cálculos finais

In [58]:
df_contratos.to_csv(os.path.join(output_dir, "contratos_2017_ministerio_saude.csv"))